In [1]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_score, recall_score

2025-08-14 10:30:46.846339: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755167447.022027      76 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755167447.073171      76 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# CONFIGURATION
# ----------------------
MODEL_PATH = "/kaggle/input/unet3scse/kaggle/working/files/modified_uavid_dataset/UnetModel.keras"  # change to your saved model path
IMG_HEIGHT = 256
IMG_WIDTH = 256
NUM_CLASSES = 8  # UAVid dataset classes

VAL_IMAGES_PATH = "/kaggle/input/uavid-v1/uavid_val/seq16/Images"
VAL_LABELS_PATH = "/kaggle/input/uavid-v1/uavid_val/seq16/Labels"

In [ ]:
# import tensorflow as tf
# from tensorflow.keras import layers as L

# class ECALayer(L.Layer):
#     """ Efficient Channel Attention (ECA) Layer """
#     def __init__(self, k_size=3, **kwargs):
#         super(ECALayer, self).__init__(**kwargs)
#         self.k_size = k_size
#         self.conv = L.Conv1D(1, kernel_size=k_size, padding="same", use_bias=False)
#         self.sigmoid = L.Activation("sigmoid")

#     def call(self, x):
#         ch_avg = L.GlobalAveragePooling2D(keepdims=True)(x)  
#         ch_avg = tf.squeeze(ch_avg, axis=[1, 2])  
#         ch_avg = tf.expand_dims(ch_avg, axis=-1)  

#         ch_avg = self.conv(ch_avg)  
#         ch_avg = self.sigmoid(ch_avg)

#         ch_avg = tf.reshape(ch_avg, [-1, 1, 1, x.shape[-1]]) 

#         eca = x * ch_avg
#         return eca 

In [ ]:
# def SpaCE(x, k_size=3):
#     """ECA-enhanced scSE (Efficient Channel Squeeze & Spatial Excitation + Spatial Squeeze & Efficient Channel Excitation)"""
    
#     # Efficient Channel Attention (ECA replaces SE)
#     eca = ECALayer(k_size)(x)

#     # Spatial Attention
#     spatial = L.Conv2D(1, kernel_size=1, activation="sigmoid")(x)
#     sSE = L.Multiply()([x, spatial])

#     # Combining both attentions
#     x_out = L.Add()([eca, sSE])
#     return x_out

In [3]:
import tensorflow as tf
from tensorflow.keras import layers as L

def scse_module(x):
    """ SCSE (Squeeze-and-Excitation + Spatial Attention) Module """
    
    # Channel Attention (SE Block)
    ch_avg = L.GlobalAveragePooling2D()(x)
    ch_avg = L.Dense(x.shape[-1] // 2, activation="relu")(ch_avg)
    ch_avg = L.Dense(x.shape[-1], activation="sigmoid")(ch_avg)
    ch_avg = L.Reshape((1, 1, x.shape[-1]))(ch_avg)
    x_ch = L.Multiply()([x, ch_avg])

    # Spatial Attention
    spatial = L.Conv2D(1, kernel_size=1, activation="sigmoid")(x)
    x_sp = L.Multiply()([x, spatial])

    # Combining both attentions
    x_out = L.Add()([x_ch, x_sp])
    return x_out

In [ ]:
# tf.keras.config.enable_unsafe_deserialization()
# custom_objects = {
#     "SpaCE": SpaCE,
#     "ECALayer": ECALayer
# }

In [4]:
tf.keras.config.enable_unsafe_deserialization()
custom_objects = {
    "scse_module": scse_module
}

In [5]:
model = tf.keras.models.load_model(MODEL_PATH, compile=False, custom_objects=custom_objects)

I0000 00:00:1755167481.002144      76 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [6]:
# DATA LOADING & PREPROCESSING
# ----------------------
def preprocess_label(label):
    # Convert RGB label mask to single-channel class index mask
    color_map = {
        (0, 0, 0): 0,         # Background clutter
        (128, 0, 0): 1,       # Building
        (128, 64, 128): 2,    # Road
        (0, 128, 0): 3,       # Tree
        (128, 128, 0): 4,     # Low vegetation
        (64, 0, 128): 5,      # Moving car
        (192, 0, 192): 6,     # Static car
        (64, 64, 0): 7        # Human
    }
    h, w, _ = label.shape
    seg_mask = np.zeros((h, w), dtype=np.uint8)
    for rgb, idx in color_map.items():
        matches = np.all(label == rgb, axis=-1)
        seg_mask[matches] = idx
    return seg_mask

In [7]:
def load_image(path):
    img = tf.keras.utils.load_img(path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img = tf.keras.utils.img_to_array(img) / 255.0
    return img

In [8]:
def load_label(path):
    lbl = tf.keras.utils.load_img(path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    lbl = tf.keras.utils.img_to_array(lbl).astype(np.uint8)
    return preprocess_label(lbl)

In [9]:
# LOAD VALIDATION DATA
# ----------------------
val_images = sorted(os.listdir(VAL_IMAGES_PATH))
val_labels = sorted(os.listdir(VAL_LABELS_PATH))

X_val = np.array([load_image(os.path.join(VAL_IMAGES_PATH, img)) for img in val_images])
y_val = np.array([load_label(os.path.join(VAL_LABELS_PATH, lbl)) for lbl in val_labels])

In [10]:
# PREDICTIONS
# ----------------------
preds = model.predict(X_val, batch_size=4)
pred_classes = np.argmax(preds, axis=-1)

I0000 00:00:1755167577.932008     122 service.cc:148] XLA service 0x7a246002f3c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1755167577.932742     122 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1755167578.459334     122 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-08-14 10:33:01.498723: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng12{k11=2} for conv (f32[4,128,128,128]{3,2,1,0}, u8[0]{0}) custom-call(f32[4,128,128,128]{3,2,1,0}, f32[128,128,3,3]{3,2,1,0}, f32[128]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]} is taking a while...
2025-08-14 1

17/18 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step

E0000 00:00:1755167610.929475     122 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1755167611.255213     122 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1755167611.927195     122 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1755167612.207566     122 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


18/18 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step 


In [ ]:
# def calculate_precision_recall_memory_safe(model, X_val, y_val, num_classes):
#     all_true = []
#     all_pred = []

#     for i in range(len(X_val)):
#         # Predict for one image
#         pred = model.predict(np.expand_dims(X_val[i], axis=0), batch_size=1, verbose=0)

#         # Convert predictions & ground truth to class indices
#         pred_class = np.argmax(pred, axis=-1).flatten()
#         true_class = y_val[i].flatten()

#         all_true.extend(true_class)
#         all_pred.extend(pred_class)

#     all_true = np.array(all_true)
#     all_pred = np.array(all_pred)

In [ ]:
# # Example usage (replace with your data)
# num_classes = 12  # UAVid has 8 classes
# per_class_precision, macro_precision, micro_precision, per_class_recall, macro_recall, micro_recall = \
#     calculate_precision_recall_memory_safe(model, X_val, y_val, num_classes)

# print("Precision per class:", per_class_precision)
# print("Macro Precision:", macro_precision)
# print("Micro Precision:", micro_precision)
# print("Recall per class:", per_class_recall)
# print("Macro Recall:", macro_recall)
# print("Micro Recall:", micro_recall)

In [11]:
# PRECISION CALCULATION
# ----------------------
y_true_flat = y_val.flatten()
y_pred_flat = pred_classes.flatten()

precision_per_class = precision_score(y_true_flat, y_pred_flat, average=None, labels=range(NUM_CLASSES))
precision_macro = precision_score(y_true_flat, y_pred_flat, average="macro")
precision_micro = precision_score(y_true_flat, y_pred_flat, average="micro")

print("Precision per class:", precision_per_class)
print("Macro Precision:", precision_macro)
print("Micro Precision:", precision_micro)

Precision per class: [0.38946756 0.8012557  0.75421993 0.67558083 0.42788402 0.31351528
 0.53919638 0.47409326]
Macro Precision: 0.5469016214560152
Micro Precision: 0.6100949968610491


In [12]:
# RECALL
# ----------------------
recall_per_class = recall_score(y_true_flat, y_pred_flat, average=None, labels=range(NUM_CLASSES))
recall_macro = recall_score(y_true_flat, y_pred_flat, average="macro")
recall_micro = recall_score(y_true_flat, y_pred_flat, average="micro")

print("Recall per class:", recall_per_class)
print("Macro Recall:", recall_macro)
print("Micro Recall:", recall_micro)

Recall per class: [0.58833982 0.7693512  0.65267389 0.4722238  0.50935764 0.22641856
 0.16141638 0.02088326]
Macro Recall: 0.42508306753244746
Micro Recall: 0.6100949968610491


In [15]:
from sklearn.metrics import f1_score

# y_true and y_pred should be flattened arrays of class indices
f1_macro = f1_score(y_true_flat, y_pred_flat, average='macro')  # Equal weight to each class
f1_micro = f1_score(y_true_flat, y_pred_flat, average='micro')  # Global calculation

print(f"Macro F1: {f1_macro:.4f}")
print(f"Micro F1: {f1_micro:.4f}")

Macro F1: 0.4407
Micro F1: 0.6101


In [17]:
def overall_accuracy(y_true, y_pred):
    correct = np.sum(y_true == y_pred)
    total = y_true.size
    return correct / total

# Example usage:
oa = overall_accuracy(y_true_flat, y_pred_flat)
print(f"Overall Accuracy: {oa:.4f}")


Overall Accuracy: 0.6101
